# The Gifting & Northern Dreaming

In [1]:
import os

import pandas as pd
import ticket_helpers
from events import (filter_by_project, filter_community_events,
                    filter_public_events, filter_school_events, load_raw_data,
                    prepare, save_files)

Set output directory

In [2]:
OUT_DIR = '../../../../docs/metrics/events/the-gifting/_data/events'

The criteria are:

* project type is 'Produce (Signature)'
* event_type contains 'Public Event -'

It's then filtered by project_name is "13 - The Gifting"

In [3]:
data = load_raw_data().pipe(filter_by_project, "13 - The Gifting")
data = data.loc[data.event_type.notna()]

Run the processing and save the files

In [4]:
data.pipe(prepare).pipe(save_files, OUT_DIR)

,actual_audience_size_number_of_participants_in_person,actual_audience_size_number_of_participants_online,airtable_id,end_date,event_name,event_type,event_unique_identifier,number_of_booked_participants,postcode_(from_venue),project_name,project_type,season,start_date,status,ticket_tailor_id,ward_(from_venue),ward_name,ward_code,audience_size
1222,2771.0,NaN,270,2023-12-31,The Gifting,['Public Event - Large Scale Outdoor Event'],13 - The Gifting: The Gifting,NaN,['LS12 1AP'],13 - The Gifting,Produce (Signature),['Season 3 - Dreaming'],2023-12-22,01 - Greenlit,"ev_3027783, ev_2994143, ev_2994144, ev_2994146...",NaN,<NA>,NaN,2771.0


Filter by public events

In [5]:
PUBLIC_EVENTS_DIR = OUT_DIR+'/public'
os.makedirs(PUBLIC_EVENTS_DIR, exist_ok=True)


data.pipe(filter_public_events).pipe(prepare).pipe(save_files, PUBLIC_EVENTS_DIR)
data

,actual_audience_size_number_of_participants_in_person,actual_audience_size_number_of_participants_online,airtable_id,end_date,event_name,event_type,event_unique_identifier,number_of_booked_participants,postcode_(from_venue),project_name,project_type,season,start_date,status,ticket_tailor_id,ward_(from_venue)
1222,2771.0,NaN,270,2023-12-31,The Gifting,['Public Event - Large Scale Outdoor Event'],13 - The Gifting: The Gifting,NaN,['LS12 1AP'],13 - The Gifting,Produce (Signature),['Season 3 - Dreaming'],2023-12-22,01 - Greenlit,"ev_3027783, ev_2994143, ev_2994144, ev_2994146...",NaN


Filter by Community events

In [6]:
# COMMUNITY_EVENTS_DIR = OUT_DIR+'/community'
# os.makedirs(COMMUNITY_EVENTS_DIR, exist_ok=True)

# data.pipe(filter_community_events).pipe(prepare).pipe(save_files, COMMUNITY_EVENTS_DIR)

Filter by Schools events

In [7]:
SCHOOLS_EVENTS_DIR = OUT_DIR+'/schools'
os.makedirs(SCHOOLS_EVENTS_DIR, exist_ok=True)


data.pipe(filter_school_events).pipe(prepare).pipe(save_files, SCHOOLS_EVENTS_DIR)

,actual_audience_size_number_of_participants_in_person,actual_audience_size_number_of_participants_online,airtable_id,end_date,event_name,event_type,event_unique_identifier,number_of_booked_participants,postcode_(from_venue),project_name,project_type,season,start_date,status,ticket_tailor_id,ward_(from_venue),ward_name,ward_code,audience_size


## Ticket Processing

In [8]:
ticket_sales = (
    ticket_helpers
        .get_tickets_for_project('13 - Closing')
        .pipe(ticket_helpers.filter_public_events)
)

In [9]:
# ticket_sales = data.pipe(filter_public_events).pipe(ticket_helpers.get_tickets_for_event)

By ward

In [10]:
ticket_sales.pipe(ticket_helpers.summarise_by_ward).reset_index().to_csv(
    os.path.join(OUT_DIR, 'tickets_by_ward.csv'), index=False)

By date

In [11]:
(
  ticket_sales
    .pipe(ticket_helpers.summarise_by_date)
    .resample('W-FRI').sum()
    .cumsum().to_csv(os.path.join(OUT_DIR, 'tickets_by_date.csv'))
)

In [12]:
def add_human_date(data):
    datetime = data.index.to_series().dt.strftime('%a %d %b, %l:%M %P').rename('human_datetime')
    return pd.concat([data, datetime], axis=1)

ticket_sales.loc[
  (ticket_sales.event_datetime > pd.to_datetime('2023-12-21')) & (ticket_sales.public_event == True)
].groupby('event_datetime').number_of_tickets.sum().sort_index().pipe(add_human_date).to_csv(f'{OUT_DIR}/tickets_by_event_datetime.csv')